In [3]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [4]:
import shutil,os
shutil.copy("/content/drive/MyDrive/Controller32/colabM1M5.zip", "colab.zip")

'colab.zip'

In [5]:
import zipfile

# Replace 'file.zip' with the name of your zip file
with zipfile.ZipFile('colab.zip', 'r') as zip_ref:
    zip_ref.extractall()
    

Import libraries

In [43]:
import matplotlib.pyplot as plt
import random, shutil, pickle, sys
import numpy as np
from collections import deque
from keras import Sequential
from keras.optimizers import RMSprop
from IPython.display import display
from PIL import Image
import pandas as pd
from keras import Model
import keras, os, glob
import tensorflow as tf
from keras.layers import Layer, Dense, Conv2D, Flatten, RepeatVector,Dropout, LayerNormalization, MultiHeadAttention, GlobalAveragePooling1D
from keras.layers import Activation, LSTM, Bidirectional , Dropout
from keras import layers
# import cv2
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import codecs
import csv
import secrets
import sqlite3
from IPython.display import display
from IPython.display import clear_output
import time
from tabulate import tabulate
import json
import datetime

The Environment

In [53]:
class ForexEnv():
    def __init__(self, m5long, m5short, m1long, m1short):
        self.forex_m5long       = m5long
        self.forex_m5short      = m5short
        self.forex_m1long       = m1long
        self.forex_m1short      = m1short
        self.dataset_directory  = "colabM1M5/episode2/"
        self.current_trade      = {}
        self.position           = None
        self.holding            = 0
        self.take_profit        = 400.0
        self.stop_loss          = 400.0
        self.stop_price         = 0.0
        self.trailing_loss      = 0.0
        self.lot_size           = 0.1
        self.pip_value          = 1.5
        self.current_step       = 4
        self.current_end        = len(self.forex_m1short)
        self.account_balance    = 1000.0
        self.balance_limit      = 950.0
        self.profit_limit       = -50
        self.done               = 0
        self.target_lookahead   = 1
        self.current_trade      = {
                                    "current_price"   : 0.0,
                                    "entry_price"   : 0.0,
                                    "stop_loss"     : 0.0,
                                    "take_profit"   : 0.0,
                                    "stop_price"    : 0.0,
                                    "profit"        : 0.0,
                                    "balance"       : self.account_balance,
                                    "highest"       : 0
                                
        }
        self.account_balance_norm = MinMaxScaler(feature_range=(0, 1))
        self.holding_norm =     MinMaxScaler(feature_range=(0, 1))
        self.profit_norm =      MinMaxScaler(feature_range=(0, 1))
        self.action_norm=   MinMaxScaler(feature_range=(0, 1))
        self.total_pips = 100
        self.init_scalers()
        
    def reset(self, dummy_dataset = False):
        self.current_trade = {
            "entry_price" : 5000.0,
            "current_price": 5000.0,
            "stop_loss":5000.0,
            "take_profit" : 200.0,
            "stop_price": 5000.0,
            "profit": 0.0,
            "balance" : self.account_balance,
            "highest"       : 0

        }
        self.position = None
        self.holding = 0
        self.current_step = 4
        self.account_balance = 1000.0
        self.done = 0
        
        return self.get_observation(self.current_step - 1 , 0, self.get_scaled_values("Account Balance", self.account_balance),0,self.holding, dummy_dataset)

    def read_image(self, image_path):

        return np.asarray(Image.open(self.dataset_directory+image_path))/255
    

    def step(self, action, dummy_dataset=False):
        reward = 0
        self.stop_price = 0.0
        
        if self.position == None:
            if action == 1:
                self.position = "long"
                self.holding = 1
                current_price = float(self.forex_m1short.iloc[self.current_step]["Ask"])
                self.current_trade = {
                    "entry_price": current_price,
                    "current_price": current_price,
                    "stop_loss": current_price - self.stop_loss,
                    "take_profit": current_price + self.take_profit,
                    "stop_price": current_price - self.stop_loss,
                    "profit": 0.0,
                    "balance": self.account_balance,
                    "highest"       : 0
                    
                }
                self.stop_price = 0.0
            elif action == 2:
                self.holding = 2
                self.position = "short"
                current_price = float(self.forex_m1short.iloc[self.current_step]["Bid"])
                self.current_trade = {
                    "entry_price": current_price,
                    "current_price": current_price,
                    "stop_loss": current_price + self.stop_loss,
                    "take_profit": current_price - self.take_profit,
                    "stop_price": current_price + self.stop_loss,
                    "profit": 0.0,
                    "balance": self.account_balance,
                    "highest"       : 0
                }
                self.stop_price = current_price - self.take_profit
            elif action == 0:

                self.current_trade = {
                    "entry_price": float(self.forex_m1short.iloc[self.current_step]["Ask"]),
                    "current_price": float(self.forex_m1short.iloc[self.current_step]["Ask"]),
                    "stop_loss": float(self.forex_m1short.iloc[self.current_step]["Ask"]),
                    "take_profit": float(self.forex_m1short.iloc[self.current_step]["Ask"]),
                    "stop_price": float(self.forex_m1short.iloc[self.current_step]["Ask"]),
                    "profit": 0.0,
                    "balance": self.account_balance,
                    "highest"       : 0
                }
                self.holding = 0
                self.position = None
                reward = -0.1
                
        elif self.position == "long":
            if action == 1:
                current_price = float(self.forex_m1short.iloc[self.current_step]["Bid"])
                self.current_trade["current_price"] = current_price
                self.current_trade["profit"] = (current_price - self.current_trade["entry_price"]) 
                self.current_trade["balance"] = self.account_balance + self.current_trade["profit"]
                if self.current_trade["profit"] > self.current_trade["highest"]:
                    self.current_trade["highest"] = self.current_trade["profit"]
                try:
                    if current_price > self.current_trade["entry_price"]:
                        reward = (current_price - self.current_trade["entry_price"]) / (self.current_trade["take_profit"] - self.current_trade["entry_price"])
                    elif self.current_trade["entry_price"] > current_price:
                        reward = -((self.current_trade["take_profit"] - current_price )/(self.current_trade["take_profit"] - self.current_trade["stop_loss"]) )
                except:
                    reward = 0
                ratio = (1/3) * self.current_trade["highest"]
                if self.current_trade["highest"] - ratio < self.current_trade["profit"]:
                    reward = -1
            elif action == 2:
                self.position = None
                self.holding = 0
                current_price = float(self.forex_m1short.iloc[self.current_step]["Bid"])
                try:              
                    if current_price > self.current_trade["entry_price"]:
                        reward = (current_price - self.current_trade["entry_price"]) / (self.current_trade["take_profit"] - self.current_trade["entry_price"])
                    elif self.current_trade["entry_price"] > current_price:
                        reward = -((self.current_trade["take_profit"] - current_price )/(self.current_trade["take_profit"] - self.current_trade["stop_loss"]) )
                except:
                    reward = 0                
                self.current_trade["profit"] = (current_price - self.current_trade["entry_price"]) 
                self.current_trade["current_price"] = current_price
                self.current_trade["balance"] = self.account_balance + self.current_trade["profit"]
                self.account_balance += self.current_trade["profit"]
                self.current_trade["stop_price"] = current_price
                self.current_trade["stop_loss"] = current_price 
                self.current_trade["take_profit"] = current_price  
            elif action == 0:
                self.position = None
                self.holding = 0
                current_price = float(self.forex_m1short.iloc[self.current_step]["Bid"])
                try:
                    if current_price > self.current_trade["entry_price"]:
                        reward = (current_price - self.current_trade["entry_price"]) / (self.current_trade["take_profit"] - self.current_trade["entry_price"])
                    elif self.current_trade["entry_price"] > current_price:
                        reward = -((self.current_trade["take_profit"] - current_price )/(self.current_trade["take_profit"] - self.current_trade["stop_loss"]) )             
                except:
                    reward = 0                 
                self.current_trade["current_price"] = current_price
                self.current_trade["profit"] =(current_price - self.current_trade["entry_price"]) 
                self.current_trade["balance"] = self.account_balance + self.current_trade["profit"]
                self.account_balance += self.current_trade["profit"]                 
                self.current_trade["stop_loss"] = current_price 
                self.current_trade["take_profit"] = current_price
                self.current_trade["stop_price"] = current_price                               
        elif self.position == "short":
            if action == 1:
                self.position = None
                self.holding = 0
                current_price = float(self.forex_m1short.iloc[self.current_step]["Ask"])
                try:               
                    if current_price > self.current_trade["entry_price"]:
                        reward = - (abs(current_price - self.current_trade["take_profit"]) / abs(self.current_trade["stop_loss"]-self.current_trade["take_profit"]))
                    elif self.current_trade["entry_price"] > current_price:
                        reward = (current_price-self.current_trade["entry_price"]) / (self.current_trade["entry_price"]-self.current_trade["stop_loss"])
                except:
                    reward = 0                
                self.current_trade["current_price"] = current_price
                self.current_trade["profit"] = (self.current_trade["entry_price"] - current_price) 
                self.current_trade["balance"] = self.account_balance + self.current_trade["profit"]
                self.account_balance += self.current_trade["profit"] 
                self.current_trade["stop_price"] = current_price
                self.current_trade["stop_loss"] = current_price 
                self.current_trade["take_profit"] = current_price

            elif action == 2:
                current_price = float(self.forex_m1short.iloc[self.current_step]["Ask"])
                self.current_trade["current_price"] = current_price
                if self.current_trade["profit"] > self.current_trade["highest"]:
                    self.current_trade["highest"] = self.current_trade["profit"]
                    
                try:
                    if current_price > self.current_trade["entry_price"]:
                        reward = - (abs(current_price - self.current_trade["take_profit"]) / abs(self.current_trade["stop_loss"]-self.current_trade["take_profit"]))
                    elif self.current_trade["entry_price"] > current_price:
                        reward = (current_price-self.current_trade["entry_price"]) / (self.current_trade["entry_price"]-self.current_trade["stop_loss"])
                except:
                    reward = 0
                
                self.current_trade["profit"] = (self.current_trade["entry_price"] - current_price) 
                self.current_trade["balance"] = self.account_balance + self.current_trade["profit"]
                ratio = (1/3) * self.current_trade["highest"]
                if self.current_trade["highest"] - ratio < self.current_trade["profit"]:
                    reward = -1

            elif action == 0:
                self.position = None
                self.holding = 0
                current_price = float(self.forex_m1short.iloc[self.current_step]["Ask"])
                try:
                    if current_price > self.current_trade["entry_price"]:
                        reward = - (abs(current_price - self.current_trade["take_profit"]) / abs(self.current_trade["stop_loss"]-self.current_trade["take_profit"]))
                    elif self.current_trade["entry_price"] > current_price:
                        reward = (current_price-self.current_trade["entry_price"]) / (self.current_trade["entry_price"]-self.current_trade["stop_loss"])
                except:
                    reward = 0                
                self.current_trade["current_price"] = current_price
                self.current_trade["profit"] = (self.current_trade["entry_price"] - current_price) 
                self.current_trade["balance"] = self.account_balance + self.current_trade["profit"]
                self.account_balance += self.current_trade["profit"] 
                self.current_trade["stop_loss"] = current_price 
                self.current_trade["take_profit"] = current_price
                self.current_trade["stop_price"] = current_price                
        self.current_step += 1
        if self.current_step >= self.current_end + 1:
            self.done = 1
        if  int(self.account_balance) < self.balance_limit or self.current_trade["profit"]< self.profit_limit:
            reward = -1
            self.done = 1
            
        info = {
            "_step": self.current_step,
            "c_trade": self.current_trade,
            "c_price": float(self.forex_m1short.iloc[self.current_step]["Bid"]) if self.position == "long" else float(self.forex_m1short.iloc[self.current_step]["Ask"])
        }
        if dummy_dataset:
            next_state, long_short_dict = self.get_observation(self.current_step + self.target_lookahead, self.get_scaled_values("Action", action),self.get_scaled_values("Account Balance", self.current_trade["balance"]), self.get_scaled_values("Profit", self.current_trade["profit"]),self.get_scaled_values("Holding", self.holding), True)
        else:
            next_state,long_short_dict = self.get_observation(self.current_step + self.target_lookahead, self.get_scaled_values("Action", action),self.get_scaled_values("Account Balance", self.current_trade["balance"]), self.get_scaled_values("Profit", self.current_trade["profit"]),self.get_scaled_values("Holding", self.holding), False)
        
        long_short_dict["reward"] = reward
        long_short_dict["action"] = self.get_scaled_values("Action", action)
        long_short_dict["done"] = self.done
        long_short_dict["info"] = info
        long_short_dict["account_balance"] = self.account_balance
        long_short_dict["balance"] = self.get_scaled_values("Account Balance", self.current_trade["balance"])
        long_short_dict["profit"] = self.get_scaled_values("Profit", self.current_trade["profit"])
        long_short_dict["now"] = self.current_step - 1
        long_short_dict["next"] = self.current_step
        long_short_dict["holding"] = self.get_scaled_values("Holding", self.holding)
        
        return next_state, long_short_dict

    def get_observation(self, time, action, account_balance, profit, holding, dummy_dataset = False):
        #m5long
        long_short_dict = {}
        
        m5long_last_image_path    =       self.forex_m5long.iloc[time-3]["image_path"]
        m5long_third_image_path   =       self.forex_m5long.iloc[time-2]["image_path"]
        m5long_second_image_path  =       self.forex_m5long.iloc[time-1]["image_path"]
        m5long_current_image_path =       self.forex_m5long.iloc[time]["image_path"]
        #save
        #m5short
        m5short_last_image_path    =       self.forex_m5short.iloc[time-3]["image_path"]
        m5short_third_image_path   =       self.forex_m5short.iloc[time-2]["image_path"]
        m5short_second_image_path  =       self.forex_m5short.iloc[time-1]["image_path"]
        m5short_current_image_path =       self.forex_m5short.iloc[time]["image_path"]
        #m1long
        m1long_last_image_path    =       self.forex_m1long.iloc[time-3]["image_path"]        
        m1long_third_image_path   =       self.forex_m1long.iloc[time-2]["image_path"]        
        m1long_second_image_path  =       self.forex_m1long.iloc[time-1]["image_path"]        
        m1long_current_image_path =       self.forex_m1long.iloc[time]["image_path"]
        #save 
        m1short_last_image_path    =       self.forex_m1short.iloc[time-3]["image_path"]        
        m1short_third_image_path   =       self.forex_m1short.iloc[time-2]["image_path"]
        m1short_second_image_path  =       self.forex_m1short.iloc[time-1]["image_path"]
        m1short_current_image_path =       self.forex_m1short.iloc[time]["image_path"]
        #create image representation for this
        if dummy_dataset == False:
            m5long_last_image         =       self.read_image(m5long_last_image_path)
            m5long_third_image        =       self.read_image(m5long_third_image_path)
            m5long_second_image       =       self.read_image(m5long_second_image_path)
            m5long_current_image      =       self.read_image(m5long_current_image_path)

            m5short_last_image         =       self.read_image(m5short_last_image_path)
            m5short_third_image        =       self.read_image(m5short_third_image_path)
            m5short_second_image       =       self.read_image(m5short_second_image_path)
            m5short_current_image      =       self.read_image(m5short_current_image_path)    
            
            m1long_last_image         =       self.read_image(m1long_last_image_path)
            m1long_third_image        =       self.read_image(m1long_third_image_path)
            m1long_second_image       =       self.read_image(m1long_second_image_path)
            m1long_current_image      =       self.read_image(m1long_current_image_path)
            
            m1short_last_image         =       self.read_image(m1short_last_image_path)
            m1short_third_image        =       self.read_image(m1short_third_image_path)
            m1short_second_image       =       self.read_image(m1short_second_image_path)
            m1short_current_image      =       self.read_image(m1short_current_image_path)          
            array8 = np.broadcast_to(np.array([account_balance, profit, holding]).reshape(1,-1).reshape((1,1,-1)),(224,224,3))
            stacked_image   =       np.stack([
                                        m5long_last_image, m5long_third_image, m5long_second_image, m5long_current_image,
                                        m5short_last_image, m5short_third_image, m5short_second_image, m5short_current_image,
                                        m1long_last_image, m1long_third_image, m1long_second_image, m1long_current_image,
                                        m1short_last_image, m1short_third_image, m1short_second_image, m1short_current_image, array8], axis=2)
            stacked_image   =       stacked_image.transpose((2,0,1,3))
            stacked_image   =       np.expand_dims(stacked_image, axis=0)
            
        state_features = [account_balance, profit, holding, action]
        long_short_dict["state_features"] = state_features
        long_short_dict["holding"] = holding
        long_short_dict["balance"] = account_balance
        long_short_dict["account_balance"] = self.account_balance
        long_short_dict["now"] = time
        long_short_dict["next"] = time + 1
        long_short_dict["reward"] = 0
        long_short_dict["action"] = action
        long_short_dict["profit"] = profit
        long_short_dict["done"] = self.done
        
        if dummy_dataset:
            return [], long_short_dict
        else:
            return stacked_image, long_short_dict
    def get_scaled_values(self, key, value):
        if key == "Account Balance":return self.account_balance_norm.transform(np.array(float(value)).reshape(-1,1))[0][0]
        elif key == "Holding":        return self.holding_norm.transform(np.array(float(value)).reshape(-1,1))[0][0]
        elif key == "Profit":         return self.profit_norm.transform(np.array(float(value)).reshape(-1,1))[0][0]
        elif key == "Action":         return self.action_norm.transform(np.array(float(value)).reshape(-1,1))[0][0]        
    
    def init_scalers(self):
        self.account_balance_norm.fit(np.array([-1000,0,5000]).reshape(-1,1))
        self.holding_norm.fit(np.array([0,2]).reshape(-1,1))
        self.profit_norm.fit(np.array([-400,0,400]).reshape(-1,1))
        self.action_norm.fit(np.array([0,2]).reshape(-1,1))

The agent

In [49]:

class Agent:
    def __init__(self, state_size, action_size, env):
        self.model_name = "v19_3D_3States"
        self.database = "v19_3D_3States"
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = 0.99
        self.alpha = 0.00025
        self.epsilon = 1.0
        self.epsilon_min = 0.1
        self.epsilon_decay = 0.995
        self.update_rate = 1000
        self.timesteps = 1
        self.target_update_rate = 0.06
        self.working_directory = "colab/"#"/content/drive/MyDrive/Controller32/colab/"
        self.model_directory = self.working_directory + "memories/"
        self.actor_model = self._build_actor_model()
        self.critic_model = self._build_critic_model()
        self.memory_indexer = 0
        #Initialize the target models with sam
        self.target_actor_model = self._build_actor_model()
        self.target_critic_model = self._build_critic_model()
        self.target_critic_model.set_weights(self.critic_model.get_weights())
        self.env = env
        self.remote = True
        self.__setup__()
        self.__epsilon_load__()
    
    def __epsilon_load__(self):
        self.connect()
        self.cursor.execute(f"SELECT epsilon FROM history WHERE   ID = (SELECT MAX(id)  FROM history)")
        row = self.cursor.fetchone()
        if row:
            self.epsilon = row[0]
        self.shutdown()
    def __setup__(self, tables = ["memory", "specialmemory", "history"]):
        for t in tables:
            self.connect()
            self.create_tables(t)
            print(f"Created {t} table")
        self.shutdown()
        print(f"Closing database")
        time.sleep(5)
    def create_tables(self, table_name):
        #to do add primary key to this table
        if table_name == "history":
            self.cursor.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                date DATE,
                name TEXT,
                memory_indexer INTEGER, 
                epsilon REAL
                )''')
        else:
            self.cursor.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                long_short_dict TEXT,
                long_short_dict_2 TEXT,
                action REAL, 
                now INTEGER,
                next INTEGER,            
                count INTEGER,
                priority INTEGER
                )''')
        self.connection.commit()

    def connect(self):
        #remote
        if self.remote:
            self.connection = sqlite3.connect(f"{self.working_directory}{self.database}.db")
        #local
        else:
            self.connection = sqlite3.connect(f"{self.working_directory}/{self.database}.db")
        self.cursor = self.connection.cursor()
    def shutdown(self):
        self.cursor.close()
        self.connection.close()

    def _build_base_model(self):
        model = Sequential()
        model.add(tf.keras.layers.Conv3D(32, (3,3,3), strides=(1,4,4), padding="same", input_shape=self.state_size))
        model.add(Activation('relu'))
        model.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same'))
        model.add(tf.keras.layers.Conv3D(64, (3,3,3), padding="same"))
        model.add(Activation('relu'))
        model.add(tf.keras.layers.MaxPooling3D(pool_size=(1,2,2), strides=(1,2,2), padding='same'))
        model.add(tf.keras.layers.Conv3D(64, (3,3,3), padding="same"))
        model.add(Activation('relu'))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(128, activation='relu'))
        model.add(Dense(64, activation='relu'))
        model.add(Dense(32, activation='relu'))
        return model
    # Build the actor model responsible for predicting
    
    def _build_actor_model(self):
        model = self._build_base_model()
        model.add(Dense(self.action_size, activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=self.alpha, rho=0.95, epsilon=None))
        return model

    
    #Build the critic model responsible for criticising and calculating the risk of the reward
    def _build_critic_model(self):
        model = self._build_base_model()
        model.add(Dense(self.action_size, activation='softmax'))
        model.add(Dense(1, activation='linear'))
        model.compile(loss='mse', optimizer=RMSprop(learning_rate=self.alpha, rho=0.95, epsilon=None))
        
        return model

    def replay_v3(self, batch_size = 128, generate_dataset = False ):
        update_weights = 0
        break_while = True
        while(break_while):
            try:
                if generate_dataset:
                    self.dataset_generator(save=True,show_image=False, show_info = False ,_clear_output = True , start = 0, end = 5 , step = 1)    
                self.connect()
                print("Relay Connected")
                self.cursor.execute(f"SELECT * FROM memory WHERE action = 0 ORDER BY RANDOM() LIMIT {batch_size*2}")
                rows = self.cursor.fetchall()
                # print("Relay Connect ed to rows for action = 0 len rows", len(rows))
                self.cursor.execute(f"SELECT * FROM memory WHERE action = 0.5 ORDER BY RANDOM() LIMIT {batch_size*2}")
                rows += self.cursor.fetchall()
                # print("Relay Connect ed to rows for action = 2 len rows", len(rows))
                self.cursor.execute(f"SELECT * FROM memory WHERE action = 1 ORDER BY RANDOM() LIMIT {batch_size*2}")
                rows += self.cursor.fetchall()                
                print("Relay Connected to rows for action = 0 len rows", len(rows))    
                self.cursor.execute(f"SELECT * FROM specialmemory WHERE action = 0 ORDER BY RANDOM() LIMIT {batch_size*2}")
                rows += self.cursor.fetchall()
                print("Relay Connected to rows for action = 0 len rows", len(rows))
                self.cursor.execute(f"SELECT * FROM specialmemory WHERE action = 0.5 ORDER BY RANDOM() LIMIT {batch_size*2}")
                rows += self.cursor.fetchall()
                self.cursor.execute(f"SELECT * FROM specialmemory WHERE action = 1 ORDER BY RANDOM() LIMIT {batch_size*2}")
                rows += self.cursor.fetchall()
                print("Relay Connected to rows for action = 0 len rows", len(rows))
                self.shutdown()

                random.shuffle(rows)
                print(f"length of rows{len(rows)}")
                for i,record in enumerate(rows):            
                    # print("records",record)
                    long_short_dict_now = json.loads(record[1])
                    long_short_dict_next = json.loads(record[2])
                    state , _ = self.env.get_observation(record[4], long_short_dict_now["action"], long_short_dict_now["balance"], long_short_dict_now["profit"], long_short_dict_now["holding"])
                    next_state, _ = self.env.get_observation(record[5], long_short_dict_next["action"], long_short_dict_next["balance"], long_short_dict_next["profit"], long_short_dict_next["holding"]) #np.expand_dims(current_stacked, axis=0)
                    
                    reward = long_short_dict_now["reward"]
                    try:
                        done = long_short_dict_now["done"]
                    except:
                        try:
                            done = long_short_dict_now["info"]["done"]
                        except:
                            print("Exception for Done not found")
                            exit(1)
                    target = reward
                    action = int(self.env.action_norm.inverse_transform([[record[3]]])[0][0])

                    headers = ["index","Variations", "Probs" ,"rewards", "Actions"]
                    rows_display = []            
                    if not done:
                        target = (reward + self.gamma * self.target_critic_model.predict(next_state)[0][0])
                    target_f = self.actor_model.predict(state)
                    row = [i, "target_f", ",".join(str("{:.2f}".format(x)) for x in list(target_f[0])), target]
                    rows_display.append(row)
                    target_f[0][action] = target
                    row = [i, "suggested",  ",".join(str("{:.2f}".format(x)) for x in list(target_f[0])), action]
                    rows_display.append(row)
                    
                    #You need to test after every 64 epochs
                    #test an episode with a certain number of lists
                    if i % 128 == 0 and i > 0 :
                        self.dataset_generator(save=False,show_image=False, show_info = True ,_clear_output = True , start = 50, end=100, step = 50)
                        self.actor_model.save_weights(self.model_directory + self.model_name +"_"+ str(self.memory_indexer+1) + "_actor.h5")
                        self.critic_model.save_weights(self.model_directory + self.model_name + "_" +str(self.memory_indexer)+ "_critic.h5")

                    self.actor_model.fit(state, target_f, epochs=1, verbose=0)
                    target_critic = np.array([[target]])
                    self.critic_model.fit(state, target_critic, epochs=1, verbose=0)

                    row = [i, "final", ",".join(str("{:.2f}".format(x)) for x in list(self.actor_model.predict(state)[0])), self.critic_model.predict(state, verbose=0)[0]]
                    rows_display.append(row)

                    print(tabulate(rows_display,headers=headers, tablefmt='grid'))

                if update_weights % 800 == 0 and update_weights > 0:
                    online_critic_weights = self.critic_model.get_weights()
                    target_critic_weights = self.target_critic_model.get_weights()
                    
                    for i in range(len(target_critic_weights)):
                        target_critic_weights[i] = self.target_update_rate * online_critic_weights[i] + (1 - self.target_update_rate) * target_critic_weights[i]
                    self.target_critic_model.set_weights(target_critic_weights)
                    self.connect()
                    self.cursor.execute("INSERT INTO history VALUES (NULL, ?, ?, ?, ?)",tuple([datetime.datetime.now(), self.model_name, self.memory_indexer, self.epsilon]))
                    self.connection.commit()
                    self.shutdown()
                    self.save_model()
                    print("\n\n\nModel Weights Updated\n\n\n")

                update_weights += 1
                self.memory_indexer += 1
                clear_output(wait=True)

                if self.epsilon >= agent.epsilon_min:
                    agent.epsilon *= agent.epsilon_decay

            except KeyboardInterrupt:
                self.shutdown()
                self.save_model()
                time.sleep(10)
                break_while = False
                rows=[]
                state_values=[]
                nextstate_values=[]
                state = []
                next_state=[]
            #     clear_output(wait=True)
            except Exception as e:
                print(f"\nException in Replay : \n {e} \n") 
                pass
                self.shutdown()
                self.save_model()
                time.sleep(10)
                break_while = False
                rows=[]
                state_values=[]
                nextstate_values=[]
                state = []
                next_state=[]
                clear_output(wait=True)
    
    def remember_v3(self, long_short_dict_now,long_short_dict_next,count = 0, priority=0, dummy_dataset = False):
        row = []
        row.append(json.dumps(long_short_dict_now))
        row.append(json.dumps(long_short_dict_next))
        row.append(long_short_dict_now["action"])
        row.append(long_short_dict_now["now"])
        row.append(long_short_dict_now["next"])
        row.append(count)
        row.append(priority)
        if dummy_dataset:
            if long_short_dict_now["reward"] > 0.02 :
                self.cursor.execute("INSERT INTO specialmemory VALUES (NULL, ?, ?, ?, ?, ?, ?, ? )",tuple(row))
                self.connection.commit()
            # else:
            #     self.cursor.execute("INSERT INTO memory VALUES (NULL, ?, ?, ?, ?, ?, ?, ? )",tuple(row))
            #     self.connection.commit()            
        else:
            self.connect()
            if long_short_dict_now["reward"] > 0.02 :
                self.cursor.execute("INSERT INTO specialmemory VALUES (NULL, ?, ?, ?, ?, ?, ?, ? )",tuple(row))
                self.connection.commit()
            # else:
            #     self.cursor.execute("INSERT INTO memory VALUES (NULL, ?, ?, ?, ?, ?, ?, ? )",tuple(row))
            #     self.connection.commit()
            self.shutdown()
        
    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return secrets.randbelow(self.action_size) if np.random.choice(20) % 2 == 0 else np.random.choice(self.action_size) , [[0,0,0]]
        else:
            pred = self.actor_model.predict(state)
            return np.argmax(pred[0]), pred   
   
    def predict(self, state):
        return self.actor_model.predict(state)

    # Load the models
    def load(self):
        self.actor_model.load_weights(self.model_directory + self.model_name + str(self.memory_indexer) + "_actor.h5")
        self.critic_model.load_weights(self.model_directory + self.model_name + str(self.memory_indexer) + "_critic.h5")
        self.target_critic_model.set_weights(self.critic_model.get_weights())
    
    # Save the models
    def save_model(self):
        self.actor_model.save_weights(self.model_directory + self.model_name + str(self.memory_indexer) + "_actor.h5")
        self.critic_model.save_weights(self.model_directory + self.model_name + str(self.memory_indexer) + "_critic.h5")

        
    def read_image(self, image_path):
        return np.asarray(Image.open(image_path))/255
    '''
        this function generates random environment noise for the ai
        it takes save parameter if you want the random data to be saved or not. 
        also this function can run the environment to see how the ai behaves. It can print out how the learning is progressing
    '''
    def dataset_generator(self, save = False, show_image=False, show_info=True, _clear_output=False, start=0, end=20, step = 2):
        print("Database generator")
        episode = 0
        for epoch in list(range(start, end, step)):
            state, long_short_dict_now = self.env.reset(dummy_dataset = True)
            headers = ["index","Action","epsilon","Pos","done","reward","E Price","C Price", "takep","stopLoss","profit" ,"balance","pred"]
            self.env.current_step = 4
            self.env.current_end = 1226
            print(f"episode {episode}")
            for index in list(range(4,1226, 1)):
                rows = [] 
                row = []    
                action, pred = secrets.randbelow(self.action_size) if np.random.choice(20) % 2 == 0 else np.random.choice(self.action_size), [[0,0,0]]#self.act(state)
                next_state, long_short_dict_next = self.env.step(action, dummy_dataset = True)
                reward = long_short_dict_next["reward"]
                if save:
                    self.remember_v3(long_short_dict_now , long_short_dict_next, 0,0)                    
                state = next_state
                if show_image:
                    self.display_row_images(long_short_dict_now, long_short_dict_next)
                long_short_dict_now = long_short_dict_next
                row = [index,action,self.epsilon,long_short_dict_now["holding"], long_short_dict_now["done"], "{:.2f}".format(reward), long_short_dict_now["info"]["c_trade"]["entry_price"],long_short_dict_now["info"]["c_trade"]["current_price"],long_short_dict_now["info"]["c_trade"]["take_profit"],long_short_dict_now["info"]["c_trade"]["stop_loss"], long_short_dict_now["info"]["c_trade"]["profit"],long_short_dict_now["balance"], ",".join(str("{:.2f}".format(x)) for x in list(pred[0]))]
                row_totals = ["_step",long_short_dict_now["info"]["_step"]-1,"","","","Total",5 * long_short_dict_now["info"]["c_trade"]["profit"],"",""]
                rows.append(row)
                rows.append(row_totals)
                if show_image:
                    self.display_row_images(long_short_dict_now, long_short_dict_next)
                if show_info:
                    print(tabulate(rows, headers=headers, tablefmt='grid'))
                if long_short_dict_now["done"] == 1 or index > 1225:
                    done=0
                    print("Episode {} ".format(episode), end="\r")
                    state, long_short_dict_now = self.env.reset()
                    clear_output(wait=True)
                    break
            episode += 1
            # print(f"Episode {episode}")
            if _clear_output:
                clear_output(wait=True)
    def display_row_images(self,long_short_dict_now , long_short_dict_next):
        imageq = Image.open(f"{self.env.dataset_directory}{self.env.forex_m1short.iloc[long_short_dict_now['now']-3]['image_path']}")
        imaget = Image.open(f"{self.env.dataset_directory}{self.env.forex_m1long.iloc[long_short_dict_now['now']-3]['image_path']}")
        images = Image.open(f"{self.env.dataset_directory}{self.env.forex_m5long.iloc[long_short_dict_now['now']-3]['image_path']}")
        imagec = Image.open(f"{self.env.dataset_directory}{self.env.forex_m5short.iloc[long_short_dict_now['now']]['image_path']}")
        # Calculate the required dimensions for the new image
        new_width = imageq.width * 4
        new_height = imageq.height
        # Create a new image with the required dimensions
        new_image = Image.new("RGB", (new_width, new_height))
        # Paste the individual images side by side
        new_image.paste(imageq, (0, 0))
        new_image.paste(imaget, (imageq.width, 0))
        new_image.paste(images, (imageq.width + imaget.width, 0))
        new_image.paste(imagec, (imageq.width + imaget.width + images.width, 0))
        # Display the final imagem5short_image_path
        
        display(new_image) 


In [50]:
# env = ForexEnv(m5long, m5short, m1long, m1short)

In [51]:
#open datasets
# m5long = pd.read_csv("/content/drive/MyDrive/Controller32/final_m5long.csv")
# m5short = pd.read_csv("/content/drive/MyDrive/Controller32/final_m5short.csv")
# m1long = pd.read_csv("/content/drive/MyDrive/Controller32/final_m1long.csv")
# m1short = pd.read_csv("/content/drive/MyDrive/Controller32/final_m1short.csv")
m5long = pd.read_csv("final_m5long.csv")
m5short = pd.read_csv("final_m5short.csv")
m1long = pd.read_csv("final_m1long.csv")
m1short = pd.read_csv("final_m1short.csv")

In [54]:
env = ForexEnv(m5long, m5short, m1long, m1short)
state_size=(17,224,224, 3)
action_size = 3
agent = Agent(state_size, action_size, env)
agent.alpha = 0.00025
agent.memory_indexer = 0
agent.epsilon = 1.0
# print(agent.critic_model.summary())
# agent.actor_model.summary()

Created memory table
Created specialmemory table
Created history table
Closing database


In [ ]:
agent.dataset_generator(save=True, show_image = False, _clear_output=True, show_info=True, start= 1, end=20, step=1)

# agent.replay_v3()




In [ ]:
agent.replay_v3(32)

1


In [32]:
def dataset_generator(agent,save = False, show_image=False, show_info=True, _clear_output=False, start=0, end=20, step = 2):
    print("Database generator")
    agent.epsilon = 0.0001
    episode = 0
    memo = 0
    for epoch in list(range(start, end, step)):
        state, long_short_dict_now = agent.env.reset(dummy_dataset = False)
        headers = ["index","Action","epsilon","Pos","done","reward","E Price","C Price", "takep","stopLoss","profit" ,"balance","pred"]
        agent.env.current_step = 4
        agent.env.current_end = 1226
        print(f"episode {episode}")
        for index in list(range(4,1226, 1)):
            rows = [] 
            row = []
            print(state.shape)
            print(agent.act(state))
            action, pred = agent.act(state)#secrets.randbelow(self.action_size) if np.random.choice(20) % 2 == 0 else np.random.choice(self.action_size), [[0,0,0]]#self.act(state)
            
            next_state, long_short_dict_next = agent.env.step(action, dummy_dataset = False)
            reward = long_short_dict_next["reward"]
            if save:
                agent.remember_v3(long_short_dict_now , long_short_dict_next, 0,0)                    
            state = next_state
            if show_image:
                agent.display_row_images(long_short_dict_now, long_short_dict_next)
            long_short_dict_now = long_short_dict_next
            row = [index,action,agent.epsilon,long_short_dict_now["holding"], long_short_dict_now["done"], "{:.2f}".format(reward), long_short_dict_now["info"]["c_trade"]["entry_price"],long_short_dict_now["info"]["c_trade"]["current_price"],long_short_dict_now["info"]["c_trade"]["take_profit"],long_short_dict_now["info"]["c_trade"]["stop_loss"], long_short_dict_now["info"]["c_trade"]["profit"],long_short_dict_now["balance"], ",".join(str("{:.2f}".format(x)) for x in list(pred[0]))]
            row_totals = ["_step",long_short_dict_now["info"]["_step"]-1,"","","","Total",5 * long_short_dict_now["info"]["c_trade"]["profit"],"",""]
            rows.append(row)
            rows.append(row_totals)
            if show_image:
                agent.display_row_images(long_short_dict_now, long_short_dict_next)
            if show_info:
                print(tabulate(rows, headers=headers, tablefmt='grid'))
            if long_short_dict_now["done"] == 1 or index > 1225:
                done=0
                print("Episode {} ".format(episode), end="\r")
                state, long_short_dict_now = agent.env.reset()
                clear_output(wait=True)
                break
                
        episode += 1
        # print(f"Episode {episode}")
        if _clear_output:
            clear_output(wait=True)
        break

In [ ]:
dir(agent.save_model())

In [ ]:
dataset_generator(agent, start=1 , end=3, step=1, show_info=True, show_image=True)

In [40]:
agent.__setup__()

Created memory table
Created specialmemory table
Created history table
Closing database


# Dummy dataset

In [ ]:
m5long = pd.read_csv("final_m5long.csv")
m5short = pd.read_csv("final_m5short.csv")
m1long = pd.read_csv("final_m1long.csv")
m1short = pd.read_csv("final_m1short.csv")

# Positive dataset generator

In [ ]:

# time.sleep(10)
agent.epsilon = 0.003013798250576# 1.0#0.0003# 0.46211964903917074#1.0#0.00996820918179746
batch_size = 32
done = False
info = {}
reward = 0
episode = 0
state, long_short_dict = env.reset(dummy_dataset=True)

headers = ["index","Action","epsilon","Pos","done","reward","E Price","C Price", "takep","highest","profit" ,"balance","pred"]
break_while = True
show_image = False
show_info = True
counter = -1

def generate_sequence(length, check=False, key=0):
    start_value = str(random.randint(0, 2))
    if check:
        start_value = key
    if start_value == 0:
        sequence = [0] * (length) 
    elif start_value == 1:
        sequence = [1] * (length - 1) + [0]
    elif start_value == 2:
        sequence = [2] * (length - 1) + [0]
    return sequence

steps = [10,20,30,40,50, 60, 70, 80, 90, 100, 110,120,130,140,150,160,170,180,190,200]
actions = [1,2]
total = list(range(4, 1226, 1))
sections = [total[i:i+200] for i in range(0, len(total), 200)]
delta = 0
break_all = False
stop_asking = True
agent.connect()
operations = 0
for _action in actions:
    if break_all:
        break
    for step in steps:
        if break_all:
            break
        sequences = generate_sequence(step, True, _action) 
        for section in sections:
            if break_all:
                break
            # print(f"length: {len(section)} : {section}\nSteps : {step}\nsequence:{sequences}\n")
            section_parts = []
            if len(sequences) == 10:
                section_parts = [section[i:i+10] for i in range(0, len(section), 10)]
            elif len(sequences) == 20:
                section_parts = [section[i:i+20] for i in range(0, len(section), 20)]
            elif len(sequences) == 30:
                section_parts = [section[i:i+30] for i in range(0, len(section), 30)]
            elif len(sequences) == 40:
                section_parts = [section[i:i+40] for i in range(0, len(section), 40)]
            elif len(sequences) == 50:
                section_parts = [section[i:i+50] for i in range(0, len(section), 50)]
            elif len(sequences) == 60:
                section_parts = [section[i:i+60] for i in range(0, len(section), 60)]                
            elif len(sequences) == 70:
                section_parts = [section[i:i+70] for i in range(0, len(section), 70)]
            elif len(sequences) == 80:
                section_parts = [section[i:i+80] for i in range(0, len(section), 80)]
            elif len(sequences) == 90:
                section_parts = [section[i:i+90] for i in range(0, len(section), 90)]
            elif len(sequences) == 100:
                section_parts = [section[i:i+100] for i in range(0, len(section), 100)]                
            elif len(sequences) == 110:
                section_parts = [section[i:i+110] for i in range(0, len(section), 110)]
            elif len(sequences) == 120:
                section_parts = [section[i:i+120] for i in range(0, len(section), 120)]
            elif len(sequences) == 130:
                section_parts = [section[i:i+130] for i in range(0, len(section), 130)]
            elif len(sequences) == 140:
                section_parts = [section[i:i+140] for i in range(0, len(section), 140)]
            elif len(sequences) == 150:
                section_parts = [section[i:i+150] for i in range(0, len(section), 150)]
            elif len(sequences) == 160:
                section_parts = [section[i:i+160] for i in range(0, len(section), 160)]
            elif len(sequences) == 170:
                section_parts = [section[i:i+150] for i in range(0, len(section), 170)]
            elif len(sequences) == 180:
                section_parts = [section[i:i+150] for i in range(0, len(section), 180)]
            elif len(sequences) == 190:
                section_parts = [section[i:i+150] for i in range(0, len(section), 190)]                
            elif len(sequences) == 200:
                section_parts = [section[i:i+200] for i in range(0, len(section), 200)] 
            print(sequences)
            
            # print(f"length: {len(section)} : {section}\nSteps : {step}\nsequence:{sequences}\n")
            # print(f"section parts {len(section_parts)}\n{section_parts}")
            # if len(section_parts) > 0 and step == 50:
            #     # print(f"\nSection parts found {section_parts} \nsection sequences {sequences}\n")
            for parts in section_parts:
                
                env = ForexEnv(m5long, m5short, m1long, m1short)
                state, long_short_dict_now = env.reset(dummy_dataset=True)
                agent.env = env
                agent.env.current_step = parts[0]
                agent.env.current_end = parts[-1]
                rows = []
                for index, part in enumerate(parts):
                    # print(index, part, sequences[index], sequences)
                    action = sequences[index]
                    try:
                        next_state, long_short_dict_next = env.step(action,dummy_dataset=True)
                        agent.remember_v3(long_short_dict_now , long_short_dict_next, 0,0, dummy_dataset=True)
                        pred = [[0.35835683, 0.30909953, 0.33254367]]
                        state = next_state
                        long_short_dict_now = long_short_dict_next
                        print(f"Operations: {operations}, {agent.env.current_step}", end="\r")
                    except Exception as e:
                        print(f"Exception caught:\n{e}\n", end = "\r")
                    operations += 1
                    # row = []
                    # row = [index,action,agent.epsilon,long_short_dict_now["holding"], long_short_dict_now["done"], "{:.2f}".format(reward), long_short_dict_now["info"]["c_trade"]["entry_price"],long_short_dict_now["info"]["c_trade"]["current_price"],long_short_dict_now["info"]["c_trade"]["take_profit"],long_short_dict_now["info"]["c_trade"]["highest"], long_short_dict_now["info"]["c_trade"]["profit"],long_short_dict_now["balance"], ",".join(str("{:.2f}".format(x)) for x in list(pred[0]))]
                    # row_totals = ["_step",long_short_dict_now["info"]["_step"]-2,"","","","Total",5 * long_short_dict_now["info"]["c_trade"]["profit"],"",""]
                    # rows.append(row)
                    # rows.append(row_totals)
                    # print(tabulate(rows, headers=headers, tablefmt='grid'))   
                
                # if stop_asking:
                #     message = input("Do you want to continue")
                #     if message == "no":
                #         break_all = True
                #     elif message == "yes":
                #         clear_output(wait=True)
                #     elif message == "stop_asking":
                #         stop_asking = False

              
                

#             # clear_output(wait=True) 
agent.shutdown()






In [39]:
agent.shutdown()

In [17]:
a = [304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403]
len(a)

100

In [18]:
len([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

500

In [ ]:
add columns to dataset highest, duration and ratio

In [56]:
agent.replay_v3(32)

Relay Connected
Relay Connected to rows for action = 0 len rows 0
Relay Connected to rows for action = 0 len rows 0
Relay Connected to rows for action = 0 len rows 128
length of rows128
1/1 [==============================] - 0s 156ms/step
+---------+--------------+----------------+-----------+
|   index | Variations   | Probs          |   rewards |
+=========+==============+================+===========+
|       0 | target_f     | 0.00,0.00,1.00 |  0.582082 |
+---------+--------------+----------------+-----------+
|       0 | suggested    | 0.00,0.58,1.00 |  1        |
+---------+--------------+----------------+-----------+
|       0 | final        | 0.00,1.00,0.00 |  0.599077 |
+---------+--------------+----------------+-----------+
1/1 [==============================] - 0s 151ms/step
+---------+--------------+----------------+-----------+
|   index | Variations   | Probs          |   rewards |
+=========+==============+================+===========+
|       1 | target_f     | 0.00,1.00

ProgrammingError: Cannot operate on a closed database.